In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import pandas as pd
import time

# 감성분석(이 글이 긍정적으로 쓰였는지, 부정적으로 쓰였는지)

total = []
for n in tqdm(range(1, 3992, 10)):
    url = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EA%B3%A0%EB%A0%A4%EB%8C%80&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=22&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={n}")
    html = BeautifulSoup(url.text)
    news = html.find("ul", class_ = 'list_news').find_all("li", class_ = 'bx')
    for i in news:
        title = i.find('a', class_ = 'news_tit')['title']
        summary = i.find("div", class_ = 'dsc_wrap').text
        total.append([title, summary])
    time.sleep(1)

In [ ]:
df = pd.DataFrame(total, columns = ['제목', '요약'])
df

,제목,요약
0,"이관영 고려대 교수, 27대 한국에너지학회장 취임",고려대는 이관영 화공생명공학과 교수가 제27대 한국에너지학회장에 취임했다고 4일 ...
1,"신현석 고려대 교수, 한국교육학회장 취임","대학교수를 포함한 교육학자 및 교사, 교육 관련기관 종사자 등 현재 6000여 명..."
2,"이관영 고려대 연구부총장, 한국에너지학회 27대 회장 취임",이관영 고려대 화공생명공학과 교수(연구부총장)가 제27대 한국에너지학회 회장으로 ...
3,"고려대안산병원 신철민 교수, 경기도지사 표창",고려대안산병원은 신철민 정신건강의학과 교수가 정신질환자를 포함한 경기도민의 정신건...
4,고려대 한국형 온라인 공개강좌 신규 4강좌 개강,고려대는 2015년 K-MOOC 선도대학으로 선정된 이후 5년간 매년 MOOC 강...
...,...,...
3995,[종합] 조민 부산대 이어 고려대도 ‘입학 취소’,조씨는 고려대 생명과학대학 환경생태공학부를 졸업한 뒤 부산대 의학전문대학원에 진학...
3996,닻 올린 서유석號 … 위기의 자본시장 안정이 숙제,◇프로필 ▶배재고등학교(1981년 졸업) ▶고려대학교 경제학과(1988년 졸업) ...
3997,"고려대의료원 메디사이언스파크, 항암신약 개발 본격 시동",고려대 메디사이언스파크는 동화바이오관(우측 건물) 내 항암신약 GMP시설 조성을 ...
3998,조민 학력 ‘고졸’ 됐다…부산대 이어 고려대도 입학취소 확정,하지만 부산대 의학전문대학원(의전원)에 이어 고려대도 그의 입학허가를 취소하면서 ...


In [ ]:
knu = pd.read_csv("knu_sentiment_lexicon.csv")
knu

,word,polarity
0,ㅡㅡ,-1
1,ㅠㅠ,-1
2,ㅠ_ㅠ,-1
3,ㅠ,-1
4,ㅜㅡ,-1
...,...,...
14849,(^-^),1
14850,(;_;),-1
14851,(-_-),-1
14852,(-;,1


In [ ]:
total = []
for t1, t2 in zip(df['제목'], df['요약']):
    t = t1 + " " + t2 # 한문장
    score = 0
    for w, p in zip(knu['word'], knu['polarity']):
        if w in t:
            score += p
    total.append(score)

In [ ]:
len(total)

4000

In [ ]:
df['점수'] = total
df

,제목,요약,점수
0,"이관영 고려대 교수, 27대 한국에너지학회장 취임",고려대는 이관영 화공생명공학과 교수가 제27대 한국에너지학회장에 취임했다고 4일 ...,-2
1,"신현석 고려대 교수, 한국교육학회장 취임","대학교수를 포함한 교육학자 및 교사, 교육 관련기관 종사자 등 현재 6000여 명...",2
2,"이관영 고려대 연구부총장, 한국에너지학회 27대 회장 취임",이관영 고려대 화공생명공학과 교수(연구부총장)가 제27대 한국에너지학회 회장으로 ...,-4
3,"고려대안산병원 신철민 교수, 경기도지사 표창",고려대안산병원은 신철민 정신건강의학과 교수가 정신질환자를 포함한 경기도민의 정신건...,-4
4,고려대 한국형 온라인 공개강좌 신규 4강좌 개강,고려대는 2015년 K-MOOC 선도대학으로 선정된 이후 5년간 매년 MOOC 강...,4
...,...,...,...
3995,[종합] 조민 부산대 이어 고려대도 ‘입학 취소’,조씨는 고려대 생명과학대학 환경생태공학부를 졸업한 뒤 부산대 의학전문대학원에 진학...,-2
3996,닻 올린 서유석號 … 위기의 자본시장 안정이 숙제,◇프로필 ▶배재고등학교(1981년 졸업) ▶고려대학교 경제학과(1988년 졸업) ...,2
3997,"고려대의료원 메디사이언스파크, 항암신약 개발 본격 시동",고려대 메디사이언스파크는 동화바이오관(우측 건물) 내 항암신약 GMP시설 조성을 ...,-2
3998,조민 학력 ‘고졸’ 됐다…부산대 이어 고려대도 입학취소 확정,하지만 부산대 의학전문대학원(의전원)에 이어 고려대도 그의 입학허가를 취소하면서 ...,1


In [ ]:
df2 = df.sort_values(by='점수')
df2

,제목,요약,점수
1383,고려대 구로병원 한창수 교수,"한편, 한창수 교수는 고려대 의대를 졸업하고, 동대학원과 미국 듀크대학교에서 학위...",-11
698,"리용호 전 北외무상 처형됐나…""본보기식 처벌 가능성""",남성욱 고려대 통일외교학부 교수는 “리 전 외무상이 처형을 당했다기보다는 교화소나...,-11
2512,"고려대의료원-선메디컬센터 선병원, 교육협력병원 추진 등 MOA","또 선메디컬센터 산하병원 소속 전공의 수련과 임상 교류 등 교육연구 협력, 고려대...",-10
1882,"사진작가 김중만, 폐렴 투병 중 별세..향년 68세",1954년생인 김중만 작가는 프랑스 니스 국립응용미술대학에서 서양화를 전공하던 중...,-10
283,"고려대의료기술지주·휴이노, 글로벌 수준 의료 인공지능 기술 고도화 위한 합작회사 설립","왼쪽부터 고려대 함병주 의학연구처장 겸 의료기술지주 대표, 휴이노 길영준 대표(사...",-10
...,...,...,...
250,"고려대, 특훈 교수 제도 시행하며 독보적 연구자 8인 임명","(문과대학 한문학과)까지 총 8인이다. 정진택 고려대 총장은 ""특훈임용 제도는 다...",10
249,"고려대, 특훈 교수 제도 시행하며 독보적 연구자 8인 임명","(문과대학 한문학과)까지 총 8인이다. 정진택 고려대 총장은 ""특훈임용 제도는 다...",10
3642,"[2022 정기전] '주장의 품격' 박무빈 앞세운 고려대, 연세대 꺾고 3년만의 정...",고려대학교 농구부는 28일 고양체육관에서 열린 연세대학교 농구부와의 2022 정기...,10
166,"고려대, 특훈 교수 제도 도입…8명 임명",(문과대학 한문학과) 등 8명이다. 정진택 고려대 총장은 “특훈임용 제도는 다양한...,11
